In [1]:
from nltk.corpus import brown
import nltk
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups 
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [2]:
with open('movie_summaries_bottom_250.txt', 'r') as myfile:
    data=myfile.readlines()
#print(summary[0])

In [3]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag


preprocess = [strip_proppers(doc) for doc in data]


In [4]:
#Paramter
NUM_TOPICS = 4
#Convert a collection of text documents to a matrix of token counts
vectorizer = CountVectorizer(
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(preprocess)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Different topic distribution in the first document
print(lda_Z[0])


C:\Users\Pranay\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


(249, 4)
[0.00254823 0.9923947  0.00252986 0.00252721]


In [5]:
def print_topics(model, vectorizer, top_n=4):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('girls', 25.99418329142536), ('father', 25.011184574082243), ('young', 24.86279698062035), ('group', 21.60950787669378)]
Topic 1:
[('evil', 41.151413607214636), ('life', 35.69518392231205), ('world', 33.2960812793761), ('time', 27.904198179306228)]
Topic 2:
[('shark', 6.4473721119665495), ('evil', 5.752983055837181), ('weapons', 5.527984029521206), ('professor', 4.308528955580412)]
Topic 3:
[('college', 11.723292768437195), ('friend', 8.798472188262101), ('young', 8.77574013058827), ('best', 7.905370709392387)]


In [6]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
text_font_size="8pt", text_color="#555555",
source=source, text_align='center')
plot = figure(plot_width=300, plot_height=300)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)


In [7]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [8]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\Pranay\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
1      41.385242        1       1  343.599976  -51.167015
0      38.272852        1       2  234.853607 -424.282471
3      15.067378        1       3  -29.513216   57.586979
2       5.274528        1       4 -138.259491 -315.528381, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
3547  Default  15.000000       shark  15.000000  30.0000  30.0000
1348  Default  43.000000        evil  43.000000  29.0000  29.0000
742   Default  11.000000     college  11.000000  28.0000  28.0000
1681  Default  24.000000       girls  24.000000  27.0000  27.0000
4393  Default   4.000000     weapons   4.000000  26.0000  26.0000
899   Default   6.000000      create   6.000000  25.0000  25.0000
2424  Default   7.000000       meets   7.000000  24.0000  24.0000
994   Default  16.000000       death  16.000000  23.0000  23.0000
563   Default   5.000000        camp   5.000000  22.0000  22.0000
3440  Default  14.000000      school  14.000000  21.0000  21.0000
4292  Default   4.000000   vengeance   4.000000  20.0000  20.0000
4130  Default  18.000000    treasure  18.000000  19.0000  19.0000
181   Default  13.000000        army  13.000000  18.0000  18.0000
3557  Default  12.000000        ship  12.000000  17.0000  17.0000
3101  Default   4.000000       quest   4.000000  16.0000  16.0000
2506  Default   7.000000       money   7.000000  15.0000  15.0000
1702  Default  13.000000        gold  13.000000  14.0000  14.0000
421   Default   6.000000       blood   6.000000  13.0000  13.0000
331   Default  11.000000      battle  11.000000  12.0000  12.0000
1495  Default  16.000000       fight  16.000000  11.0000  11.0000
1613  Default  19.000000      friend  19.000000  10.0000  10.0000
3034  Default   3.000000   professor   3.000000   9.0000   9.0000
4267  Default   7.000000        uses   7.000000   8.0000   8.0000
100   Default   7.000000       alive   7.000000   7.0000   7.0000
132   Default  11.000000     ancient  11.000000   6.0000   6.0000
3420  Default  24.000000        save  24.000000   5.0000   5.0000
2461  Default   4.000000    military   4.000000   4.0000   4.0000
3823  Default   3.000000      street   3.000000   3.0000   3.0000
3843  Default   9.000000     student   9.000000   2.0000   2.0000
151   Default   5.000000   apartment   5.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
2028   Topic4   0.424918  infiltrate   1.093774   1.9968  -7.2837
4393   Topic4   1.809441     weapons   4.800409   1.9666  -5.8349
4292   Topic4   1.312495   vengeance   4.372970   1.7388  -6.1559
3101   Topic4   1.239732       quest   4.131890   1.7384  -6.2130
1877   Topic4   0.712633     history   2.097620   1.8627  -6.7667
536    Topic4   0.989963      buried   3.215890   1.7641  -6.4380
3992   Topic4   0.667022        teen   2.048371   1.8203  -6.8328
1600   Topic4   0.996825        free   3.643611   1.6461  -6.4311
3278   Topic4   0.665151   resources   2.045362   1.8190  -6.8356
467    Topic4   0.662684       bound   2.062937   1.8067  -6.8393
3482   Topic4   0.973908       seeks   3.744580   1.5955  -6.4543
899    Topic4   1.286851      create   6.326463   1.3497  -6.1757
3547   Topic4   2.110378       shark  15.635392   0.9396  -5.6810
2461   Topic4   1.001112    military   4.712048   1.3933  -6.4268
2424   Topic4   1.247761       meets   7.530134   1.1447  -6.2065
1348   Topic4   1.883088        evil  43.011181  -0.1863  -5.7950
421    Topic4   0.940039       blood   6.765352   0.9686  -6.4897
2506   Topic4   0.955045       money   7.701162   0.8549  -6.4739
4267   Topic4   0.952659        uses   7.734917   0.8480  -6.4764
100    Topic4   0.940104       alive   7.329027   0.8887  -6.4896
132    Topic4   1.005307     ancient  11.387879   0.5150  -6.4226
2547   Topic4   0.752602    murdered   2.9619